In [1]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
plt.style.use(style='seaborn')
%matplotlib inline
import time

from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model #for lasso
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection 


In [50]:
df = pd.read_csv('./Data/london_merged.csv')

In [51]:
df.sample(5)

,timestamp,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season
5631,2015-08-27 06:00:00,685,15.0,15.0,82.0,14.0,2.0,0.0,0.0,1.0
6134,2015-09-17 13:00:00,1402,17.0,17.0,54.0,22.0,2.0,0.0,0.0,2.0
4007,2015-06-20 10:00:00,1571,17.0,17.0,64.0,7.0,3.0,0.0,1.0,1.0
5750,2015-09-01 05:00:00,116,15.0,15.0,85.0,11.0,3.0,0.0,0.0,2.0
1644,2015-03-13 14:00:00,1164,11.0,11.0,44.0,26.0,1.0,0.0,0.0,0.0


In [52]:
new_df = df.copy()
#adding 25% features to wind speed
#new_df['wind_speed'] = new_df.apply(lambda row: row['wind_speed'] + np.random.random_sample()*1000, axis=1 )
#adding 25% features to temp 
#new_df['t2']  = new_df.apply(lambda row: row['t2'] + np.random.random_sample()*2, axis=1)

df = df.append(new_df.sample(int(new_df.shape[0]/4)))

#enhancing data
df['year'] = df['timestamp'].apply(lambda row: row[:4])
df['month'] = df['timestamp'].apply(lambda row: row[5:7])
df['hour'] = df['timestamp'].apply(lambda row: row[11:13])

df = df.drop(['timestamp'], axis =1)
df.sample(10)

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
6577,70,16.0,16.0,94.0,12.0,7.0,0.0,0.0,2.0,2015,10,01
284,591,6.5,1.5,57.0,32.0,1.0,0.0,0.0,3.0,2015,01,21
6996,1535,13.5,13.5,65.0,10.0,4.0,0.0,0.0,2.0,2015,10,19
14382,476,18.5,18.5,83.0,10.0,3.0,1.0,0.0,1.0,2016,08,01
3311,1970,11.0,11.0,77.0,13.0,1.0,0.0,0.0,0.0,2015,05,07
10035,230,4.0,0.0,75.0,18.0,4.0,0.0,1.0,3.0,2016,02,08
16194,46,7.5,5.5,87.0,9.0,2.0,0.0,0.0,2.0,2016,11,04
12241,585,15.0,15.0,68.0,23.5,1.0,1.0,0.0,0.0,2016,05,22
6951,592,15.0,15.0,88.0,14.0,4.0,0.0,0.0,2.0,2015,10,22
2335,2713,13.5,13.5,37.0,26.0,1.0,0.0,1.0,0.0,2015,04,15


In [5]:
df['season'].unique()

array([3., 0., 1., 2.])

In [60]:
#generating data
def data_enhancement(data):
    
    gen_data = df.copy()
    for season in data['season'].unique():
        seasonal_data = gen_data[ gen_data['season'] == season]
        
        t1_std   = seasonal_data['t1'].std()
        t2_std   = seasonal_data['t2'].std()
        hum_std  = seasonal_data['hum'].std()
        wind_std = seasonal_data['wind_speed'].std()

        for i in range(seasonal_data.shape[0]):
            if np.random.randint(2) == 1: #losuje liczbe od 0 do 1  
                seasonal_data['t1'].values[i] += t1_std
            else:
                seasonal_data['t1'].values[i] -= t1_std   

            if np.random.randint(2) == 1:  
                seasonal_data['t2'].values[i] += t2_std
            else:
                seasonal_data['t2'].values[i] -= t2_std   
        
            if np.random.randint(2) == 1:   
                seasonal_data['hum'].values[i] += hum_std
            else:
                seasonal_data['hum'].values[i] -= hum_std

            if np.random.randint(2) == 1:  
                seasonal_data['wind_speed'].values[i] += wind_std
            else:
                seasonal_data['wind_speed'].values[i] -= wind_std
        
    return gen_data

gen = data_enhancement(df)                         


In [34]:
gen

,cnt,t1,t2,hum,wind_speed,weather_code,is_holiday,is_weekend,season,year,month,hour
0,182,3.0,2.000000,93.0,6.000000,3.0,0.0,1.0,3.0,2015,01,00
1,138,3.0,2.500000,93.0,5.000000,1.0,0.0,1.0,3.0,2015,01,01
2,134,2.5,2.500000,96.5,0.000000,1.0,0.0,1.0,3.0,2015,01,02
3,72,2.0,2.000000,100.0,0.000000,1.0,0.0,1.0,3.0,2015,01,03
4,47,2.0,0.000000,93.0,6.500000,1.0,0.0,1.0,3.0,2015,01,04
...,...,...,...,...,...,...,...,...,...,...,...,...
238,340,5.0,1.135830,70.0,952.981578,1.0,0.0,0.0,3.0,2015,01,23
936,63,4.0,4.318563,75.0,306.405440,4.0,0.0,0.0,3.0,2015,02,02
17186,3665,14.0,14.987899,77.0,125.733813,3.0,0.0,1.0,3.0,2016,12,12
6020,1539,17.5,17.594161,57.5,147.351105,2.0,0.0,1.0,2.0,2015,09,19


In [61]:
cat_vars = ['weather_code', 'is_holiday','is_weekend','season','year','month']
num_vars = ['t1','t2','hum','wind_speed']
num_vars

['t1', 't2', 'hum', 'wind_speed']

In [55]:
y = df['cnt']
x = df.drop(['cnt'], axis=1)


In [70]:
x_train, x_val , y_train, y_val = model_selection.train_test_split(x,y ,test_size = 0.2 , random_state = 0)

#generating extar samples 

extra_sample = gen.sample(gen.shape[0] // 3 ) # generating 30% new data

#concatinating extra sample to training data
x_train = pd.concat([x_train, extra_sample.drop(['cnt'], axis=1)])
y_train = pd.concat([y_train, extra_sample['cnt']])

#normalazing data labels
transformer = preprocessing.PowerTransformer()
y_train = transformer.fit_transform(y_train.values.reshape(-1,1))
y_val = transformer.transform(y_val.values.reshape(-1,1))

rang = abs(y_train.max()) + abs(y_train.min())

#applying imputer to fill missing num & cat data

num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value=-9999)),
])


cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder())
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars


#dictionary of all algorithms that I am going to test 
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}




In [71]:
#putting everything together

tree_classifiers = {name: pipeline.make_pipeline(tree_prepo, model) for name, model in tree_classifiers.items()}

results = pd.DataFrame({'Model': [], 'MSE': [], 'MAB': [], " % error": [], 'Time': []})

for model_name, model in tree_classifiers.items():
    
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "MSE": metrics.mean_squared_error(y_val, pred),
                              "MAB": metrics.mean_absolute_error(y_val, pred),
                              " % error": metrics.mean_squared_error(y_val, pred) / rang,
                              "Time":     total_time},
                              ignore_index=True)
### END SOLUTION


results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')

print(results_ord)


print(y_train.max())
print(y_train.min())
print(y_val[3])
print(tree_classifiers['Random Forest'].predict(x_val)[3])

ValueError: Unknown label type: 'continuous'

In [74]:
""" ALL IN ONE CELL"""

import numpy    as np
from numpy.testing._private.utils import decorate_methods
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl
import time

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config

from sklearn.tree          import DecisionTreeRegressor
from sklearn.ensemble      import RandomForestRegressor
from sklearn.ensemble      import ExtraTreesRegressor
from sklearn.ensemble      import AdaBoostRegressor
from sklearn.ensemble      import GradientBoostingRegressor
from xgboost               import XGBRegressor
from lightgbm              import LGBMRegressor
from catboost              import CatBoostRegressor

data = pd.read_csv(r'Data/london_merged.csv')

np.random.seed(0)

#target = data['cnt']
#data = data.drop(['cnt'], axis=1)

#Print data shape
#print(target.shape)
#print(data.shape)

#Take a look at nulls 0 nulls
#print(target.isnull().sum())
#print(data.isnull().sum())

#lets create a 2 new feautures
# Hour time stamp contains the year and the month,
# we will create different columns for each one

data['year'] = data['timestamp'].apply(lambda row: row[:4])
data['month'] = data['timestamp'].apply(lambda row: row.split('-')[2][:2] )
data['hour'] = data['timestamp'].apply(lambda row: row.split(':')[0][-2:] )
'''
print(data['year'])
print(data['month'])
print(data['hour'])
'''
data.drop('timestamp', axis=1, inplace=True)

#print(data.shape)


def data_enhancement(data):
    
    gen_data = data.copy()
    
    for season in data['season'].unique():
        seasonal_data =  gen_data[gen_data['season'] == season]
        hum_std = seasonal_data['hum'].std()
        wind_speed_std = seasonal_data['wind_speed'].std()
        t1_std = seasonal_data['t1'].std()
        t2_std = seasonal_data['t2'].std()
        
        for i, d in enumerate(data[data['season'] == season]):
            if np.random.randint(2) == 1:
                seasonal_data['hum'].values[i] += hum_std
            else:
                seasonal_data['hum'].values[i] -= hum_std
                
            if np.random.randint(2) == 1:
                seasonal_data['wind_speed'].values[i] += wind_speed_std
            else:
                seasonal_data['wind_speed'].values[i] -= wind_speed_std
                
            if np.random.randint(2) == 1:
                seasonal_data['t1'].values[i] += t1_std
            else:
                seasonal_data['t1'].values[i] -= t1_std
                
            if np.random.randint(2) == 1:
                seasonal_data['t2'].values[i] += t2_std
            else:
                seasonal_data['t2'].values[i] -= t2_std
        
    return gen_data

gen = data_enhancement(data)

#print(gen.shape)

#final_data = data
y = data['cnt']
x = data.drop(['cnt'], axis=1)



#print(data.shape)




cat_vars = ['season','is_weekend','is_holiday','year','month','weather_code']
num_vars = ['t1','t2','hum','wind_speed']


x_train, x_val, y_train, y_val = model_selection.train_test_split(x, y,
                                    test_size=0.2,
                                    random_state=0  # Recommended for reproducibility
                                )



extra_sample = gen.sample(gen.shape[0] // 2)
x_train = pd.concat([x_train, extra_sample.drop(['cnt'], axis=1 ) ])
y_train = pd.concat([y_train, extra_sample['cnt'] ])


transformer = preprocessing.PowerTransformer()
y_train = transformer.fit_transform(y_train.values.reshape(-1,1))
y_val = transformer.transform(y_val.values.reshape(-1,1))



rang = abs(y_train.max()) + abs(y_train.min())

num_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value=-9999)),
])

cat_4_treeModels = pipeline.Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('ordinal', preprocessing.OrdinalEncoder()) # handle_unknown='ignore' ONLY IN VERSION 0.24
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_classifiers = {
  "Decision Tree": DecisionTreeRegressor(),
  "Extra Trees":   ExtraTreesRegressor(n_estimators=100),
  "Random Forest": RandomForestRegressor(n_estimators=100),
  "AdaBoost":      AdaBoostRegressor(n_estimators=100),
  "Skl GBM":       GradientBoostingRegressor(n_estimators=100),
  "XGBoost":       XGBRegressor(n_estimators=100),
  "LightGBM":      LGBMRegressor(n_estimators=100),
  "CatBoost":      CatBoostRegressor(n_estimators=100),
}
### END SOLUTIONv

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

results = pd.DataFrame({'Model': [], 'MSE': [], 'MAB': [], " % error": [], 'Time': []})

for model_name, model in tree_classifiers.items():
    
    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
        
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "MSE": metrics.mean_squared_error(y_val, pred),
                              "MAB": metrics.mean_absolute_error(y_val, pred),
                              " % error": metrics.mean_squared_error(y_val, pred) / rang,
                              "Time":     total_time},
                              ignore_index=True)
### END SOLUTION


results_ord = results.sort_values(by=['MSE'], ascending=True, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['MSE', 'MAE'], vmin=0, vmax=100, color='#5fba7d')

print(results_ord)


print(y_train.max())
print(y_train.min())
print(y_val[3])
print(tree_classifiers['Random Forest'].predict(x_val)[3])

C:\Users\Wojtek\anaconda3\envs\dl\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Wojtek\anaconda3\envs\dl\lib\site-packages\sklearn\pipeline.py:346: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)
C:\Users\Wojtek\anaconda3\envs\dl\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Wojtek\anaconda3\envs\dl\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d arra